# Import Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

In [2]:
from tensorflow.keras.layers import Dense,Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model

In [3]:
import tensorflow.keras.backend as K
K.clear_session()

# Image Proprocessing

In [4]:
image_height, image_width = (412, 412)
batch_size = 12

train_data_dir = 'train/'
test_data_dir = 'test/'

In [5]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=0.3,
                                   shear_range=0.3,
                                   validation_split=0.4)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

Found 6541 images belonging to 3 classes.


In [6]:
test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(image_height, image_width),
    batch_size=1,
    class_mode='categorical',
    subset='validation')

Found 1078 images belonging to 3 classes.


# Load Resnet 50 Model

In [7]:
base_model = ResNet50(include_top=False, weights='imagenet')

In [8]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(test_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable=False

In [9]:
x, y = train_generator.next()

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Model Callbacks - Tensorboard and EarlyStopping

In [11]:
%load_ext tensorboard

In [12]:
import datetime
from tensorflow.keras.callbacks import TensorBoard


# Set Folder for saving logs
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                min_delta=0, 
                                patience=0, 
                                verbose=0, 
                                mode='auto', 
                                baseline=None, 
                                restore_best_weights=False)

# Model Training

In [14]:
steps_per_epoch =  train_generator.n // batch_size
validation_steps= test_generator.n //batch_size

In [15]:
model.fit(train_generator, epochs=3, callbacks=[tensorboard_callback], validation_data=test_generator, validation_steps=validation_steps)

C:\Users\Eashan\.conda\envs\fruit\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/3
546/546 [==============================] - 187s 332ms/step - loss: 0.1175 - accuracy: 0.9688 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 2/3
546/546 [==============================] - 165s 302ms/step - loss: 0.0285 - accuracy: 0.9917 - val_loss: 5.3724e-04 - val_accuracy: 1.0000
Epoch 3/3
546/546 [==============================] - 169s 309ms/step - loss: 0.0093 - accuracy: 0.9980 - val_loss: 1.4535e-05 - val_accuracy: 1.0000


In [16]:
from keras.models import save_model

model.save('my_model.h5')

In [17]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 6840), started 3:11:40 ago. (Use '!kill 6840' to kill it.)

In [ ]:
# import shutil

# shutil.rmtree('logs', ignore_errors=False)